<a href="https://colab.research.google.com/github/jiwon-hae/LLama-training/blob/main/Llama3_Fine_tuning_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요 Library install
!pip install transformers==4.39.2 peft==0.10.0 trl==0.8.6 bitsandbytes==0.43.0 accelerate==0.29.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# HF token 설정
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# Google Drive Import
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
fine_tuned_model_path = "/content/drive/MyDrive/fine_tune_output/checkpoint-50"

In [5]:
# 모델 경량화: Quantization 설정
from transformers import BitsAndBytesConfig
import torch

quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

In [6]:
# 기본 Llama 3 모델 로드
from transformers import AutoModelForCausalLM
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    quantization_config = quantization_config,
    device_map = {"": 0}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [7]:
# Fine Tune 된 모델 로드
fine_tuned_model_cp_50 = AutoModelForCausalLM.from_pretrained(
    fine_tuned_model_path,
    quantization_config=quantization_config,
    device_map = {"": 0}
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
# Tokenizer 로드
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# Prompt/Response Format 관련 설정
EOS_TOKEN = tokenizer.eos_token

def convert_to_alpaca_format(instruction, response):
    alpaca_format_str = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.\
    \n\n### Instruction:\n{instruction}\n\n### Response:\n{response}\
    """

    return alpaca_format_str

In [10]:
def test_model(instruction_str, model):
    inputs = tokenizer(
    [
        convert_to_alpaca_format(instruction_str,"",)
    ], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True, temperature = 0.05, top_p = 0.95)
    return(tokenizer.batch_decode(outputs)[0])

In [11]:
questions = [
    "List three ways to reduce plastic waste in daily life.",
    "Write a haiku about artificial intelligence",
    "How can I improve my public speaking skills?",
    "AI 분야에서 사용하는 LLM이라는 용어가 뭔지 설명해줘",
    "What is a famous tall tower in Paris?",
    "What is Fine-Tuning?",
    "15와 25의 최소공배수를 구하시오.",
    "제2차 세계대전의 주요 원인은 무엇이었나요?",
    "광합성 과정을 간단히 설명해주세요.",
    "셰익스피어의 '햄릿'에서 주인공의 성격을 분석해보세요.",
    "기후 변화가 전 세계적으로 미치는 영향에 대해 설명해주세요.",
    "파이썬에서 리스트와 튜플의 차이점은 무엇인가요?",
    "인공지능은 의식을 가질 수 있을까요? 그 이유는 무엇인가요?",
    "'안녕하세요, 오늘 날씨가 좋네요'를 영어로 번역해주세요.",
    "우주 여행을 주제로 한 짧은 이야기를 만들어보세요.",
    "인플레이션이 경제에 미치는 영향을 설명해주세요."
]

In [12]:
answers_dict = {
  "base_model_answers": [],
  "fine_tuned_model_answers": []
}
for idx, question in enumerate(questions):
    print(f"Processing EXAMPLE {idx}==============")
    base_model_output = test_model(question, base_model)
    answers_dict['base_model_answers'].append(base_model_output)
    fine_tuned_model_output = test_model(question, fine_tuned_model_cp_50)
    answers_dict['fine_tuned_model_answers'].append(fine_tuned_model_output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing EXAMPLE 0==============


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing EXAMPLE 1==============


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing EXAMPLE 14==============


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing EXAMPLE 15==============


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [13]:
def simple_format(text, width=120):
    return '\n'.join(line[i:i+width] for line in text.split('\n') for i in range(0, len(line), width))


for idx, question in enumerate(questions):
    print(f"EXAMPLE {idx}==============")
    print(f"Question: {question}")

    print("<<Base Model 답변>>")
    base_model_answer = answers_dict["base_model_answers"][idx].split("### Response:")[1]
    print(simple_format(base_model_answer))
    print("---")
    print("<<Fine Tuning Model 답변>>")
    fine_tuned_model_answer = answers_dict["fine_tuned_model_answers"][idx].split("### Response:")[1]
    print(simple_format(fine_tuned_model_answer))
    print()

EXAMPLE 0==============
Question: List three ways to reduce plastic waste in daily life.
<<Base Model 답변>>
     1. Reduce the use of plastic bags. 
     2. Reduce the use of plastic bottles. 
     3. Reduce the use of plastic straws. 
<|end_of_text|>
---
<<Fine Tuning Model 답변>>
     1. Reduce the use of single-use plastics: One of the most effective ways to reduce plastic waste is to reduce the u
se of single-use plastics, such as plastic bags, straws, and utensils. Instead, opt for reusable alternatives, such as c
loth bags, metal or bamboo straws, and reusable utensils.     2. Recycle plastic: Recycling plastic is another way to re
duce waste. By recycling plastic, it can be reused to make new products, reducing the need for new plastic to be produce
d.     3. Avoid overpackaging: Many products are overpackaged, with unnecessary amounts of plastic used

EXAMPLE 1==============
Question: Write a haiku about artificial intelligence
<<Base Model 답변>>
    <|end_of_text|>
---
<<Fine Tuni